### Section 1. Import libraries ###

In [11]:
import os
import numpy as np
import rasterio as rio
import geopandas as gpd
from sys import exit
import geospatial_functions.geospatial_analysis as ga
from osgeo import gdal
from rasterio.warp import Resampling
import pandas as pd
import matplotlib.pyplot as plt 

print('Done')

Done


### Section 2. Specify file paths ###

In [12]:
# ================ INPUTS AND CONFIGURATIONS (EDITION NEEDED) ================ 
# --- PART 1. common input files ---
root_dir = '/glade/u/home/hongli/scratch/2020_06_02HRUcomplexity/discretize'
# root_dir='/Users/hongli/Documents/proj/2020_06_01HRUcomplexity/discretize'
source_data_dir = os.path.join(root_dir, 'source_data')

huc12_shp = os.path.join(source_data_dir, 'west_huc12/WEST.huc12.shp')
huc12_field   = 'HUC12' #'HUC12int'     
Tohuc12_field = 'ToHUC'

dem_raster = os.path.join(source_data_dir, 'MERIT_Hydro_dem_NLDAS.tif') # large-domain DEM raster covering the case study area. Used for clipping.
dem_nodatavalue = -9999                                                 # the value for pixels to be considered as NoData. Obtained by checking dem_raster.

lc_raster = os.path.join(source_data_dir, 'nldas_landcover.tif')        # silimar with dem_raster, but for landcover.
lc_nodatavalue = 255                                                    # silimar with dem_nodatavalue.

soil_raster = os.path.join(source_data_dir, 'usda_mode_soilclass_vCompressed_NA_250m_ll.tif') # silimar with dem_raster, but for soil.
soil_nodatavalue = np.nan                                               # silimar with dem_nodatavalue.
    
# ---- PART 2. case study relevant inputs and configurations ----
case = 'shoshone'                                                # user-specified case study name. Used to create a case study foler to store all the case study relevant files.
outlet_hucid = '100800120304'                                    # huc12id of the outlet HUC12 of the case study area.
# case = 'tuolumne'
# outlet_hucid = '180400090504'

case_dir = os.path.join(root_dir, case)                          # case study directory. Used to store all the case study relevant files.
if not os.path.exists(case_dir): os.makedirs(case_dir)           # create case study folder if it doesn't exist.
rad_raster = os.path.join(case_dir, 'step9_merge_raw_Sw/sw.tif') # solar radiation raster for case study. Calcualted based on the reference: Allen et al., 2006. Agricultural and Forest Meteorology. 

# ---- PART 3. define GRU and HRU field names and data types ----
# note: Some avaialble dtypes for rasterio: 'int16', 'int32', 'float32', 'float64'. No 'int64'!
# reference: https://test2.biogeo.ucdavis.edu/rasterio/_modules/rasterio/dtypes.html
gruNo_field = 'gruNo'       # field name of gru number, e.g.,1,2,3...
gruNo_field_dtype = 'int32' # used to save gruNo raster. 
gruName_field = 'gruId'     # field name of gru name, e.g., 100800120101. 

hruNo_field = 'hruNo'       # field name of hru number, e.g.,1,2,3...
hruNo_field_dtype = 'int32' # used to save hruNo raster. 
hruName_field = 'hruId'     # field name of hru name, e.g., 10080012010101, 100800120102. 
hruArea_field = 'areaSqm'   # field name of hru area, used in small HRU elimination

# ---- PART 4. define common projection, nodata value, reference raster ----
proj4="+proj=aea +lat_1=20 +lat_2=60 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m no_defs"
dst_crs = rio.crs.CRS.from_string(proj4)
## Albers Equal Area Conic Projection. 
## reference 1: https://gisgeography.com/conic-projection-lambert-albers-polyconic/
## reference 2: https://epsg.io/102008

nodatavalue = -9999   # used in raster generatation

# ---- PART 5. define HRU elimination threshold (two options: value, fraction) ----
# hru_thld_type = 'value'
# hru_thld = 10**6 #1km2

hru_thld_type = 'fraction'
hru_thld = 0.05  # partial of the gru area


# ================ INTERMEDIATE FILES (NO EDITION NEEDED) ================ 
dem_raster_prj = os.path.join(os.path.dirname(dem_raster), os.path.basename(dem_raster).split('.tif')[0]+'_prj.tif')
lc_raster_prj = os.path.join(os.path.dirname(lc_raster), os.path.basename(lc_raster).split('.tif')[0]+'_prj.tif')
soil_raster_prj = os.path.join(os.path.dirname(soil_raster), os.path.basename(soil_raster).split('.tif')[0]+'_prj.tif')

dem_crop = os.path.join(case_dir, 'dem_crop.tif')               # DEM raster of case study. Cropped from the projected large-domain DEM raster.
refraster = dem_crop                                            # reference raster, used in vector rasterization and resample.
dem_crop_buf = os.path.join(case_dir, 'dem_crop_buf.tif')       # buffered DEM raster of case study. 
dem_class_raster_basename = os.path.join(case_dir, 'dem_class') # basename for DEM class files (e.g., 0:low elevation. 1: high elevation).
dem_value_raster_basename = os.path.join(case_dir, 'dem_value') # basename for DEM value files (e.g., average DEM per class).

slope_raster = os.path.join(case_dir, 'slope.tif')             # slope raster, calcualted based on dem_crop.
aspect_raster = os.path.join(case_dir, 'aspect.tif')           # aspect raster, calcualted based on dem_crop.

lc_crop = os.path.join(case_dir, 'landcover_crop.tif')         # landcover raster of case study. Cropped from the projected large-domain landcover raster.
lc_crop_resample = os.path.join(case_dir, 'landcover_crop_resample.tif') # resampled landcover raster according to the layout of refraster. 
lc_class_raster = os.path.join(case_dir, 'landcover_class.tif')# landcover class raster (e.g., canopy, non-canopy).

soil_crop = os.path.join(case_dir, 'soil_crop.tif')         # soil raster of case study.

gru_shp = os.path.join(case_dir, 'huc12.shp')               # GRU shapefile of case study. Cropped from the large-domain HUC12 shapefile.
huc12_list_txt = 'huc12Ids.txt'                                    # list of HUC12 ids.
gru_shp_prj = os.path.join(case_dir, 'gru_prj.shp')         # project GRU shapefile.
gru_raster = os.path.join(case_dir, 'gru.tif')              # project GRU raster file.
gru_corr_txt=os.path.join(case_dir, 'gruNo_HUC12_corr.txt') # correspondence between HUC12 and gru number (for recrod).
 
rad_class_raster_basename = os.path.join(case_dir, 'rad_class') # basename for radiation class files (e.g., 0:low. 1:high).
rad_value_raster_basename = os.path.join(case_dir, 'rad_value') # basename for radiation value files (e.g., average radiation per class).

print('Done')

Done


### Section 3. Generate HRU at different complexity levels ###

In [13]:
level_num = 4 #3
for level in range(1+level_num):
# for level in range(3):

    print('--- Complexity level %d ---' %(level))
    
    #  --- PART 1. define hru complexity level dependent files --- 
    print('PART 1. define files')
    hru_str = 'hru'+'_lev' + str(level)
    hru_elmn_str = hru_str+'_elmn'       
    hru_raster = os.path.join(case_dir, hru_str+'.tif')            # original HRU
    hru_vector = os.path.join(case_dir, hru_str+'.shp')
    hru_raster_elmn = os.path.join(case_dir, hru_elmn_str+'.tif')  # simplified HRU
    hru_vector_elmn = os.path.join(case_dir, hru_elmn_str+'.shp')    

    dem_classif_trigger = 300 # Elvation difference value of triggering classification.
    dem_bins = 'median'
    dem_class_raster=dem_class_raster_basename+'_lev'+str(level)+'.tif'
    dem_value_raster=dem_value_raster_basename+'_lev'+str(level)+'.tif'
    
    rad_classif_trigger = None
    rad_bins = 'median'
    rad_class_raster=rad_class_raster_basename+'_lev'+str(level)+'.tif'
    rad_value_raster=rad_value_raster_basename+'_lev'+str(level)+'.tif'
    
    # level 0: GRU = HRU 
    if level == 0: 
        # define hru discretization files
        raster_list = [gru_raster]
        raster_fieldname_list = [gruNo_field]

    # level 1: use only elevation bands in HRU generation
    if level == 1: 
        # (1) classify elevation raster per gru
        ga.classify_continuous_raster(dem_crop, gru_raster, dem_classif_trigger, dem_bins, 
                                      dem_class_raster, dem_value_raster, nodatavalue)        
        # (2) define hru discretization files
        raster_list = [gru_raster,dem_class_raster]
        raster_fieldname_list = [gruNo_field,'elevClass']
    
    # level 2: use elevation bands and landcover classes in HRU generation
    elif level == 2: 
        # (1) classify elevation raster per gru
        ga.classify_continuous_raster(dem_crop, gru_raster, dem_classif_trigger, dem_bins, 
                                      dem_class_raster, dem_value_raster, nodatavalue)        
        # (2) define hru discretization files
        raster_list = [gru_raster, dem_class_raster, lc_class_raster]
        raster_fieldname_list = [gruNo_field, 'elevClass', 'lcClass']
    
     # level 3: use elevation bands, radiation bands, landcover classes in HRU generation
    elif level == 3:
        # (1) classify elevation raster per gru
        ga.classify_continuous_raster(dem_crop, gru_raster, dem_classif_trigger, dem_bins, 
                                      dem_class_raster, dem_value_raster, nodatavalue)        
        # (2) classify radiation raster per gru
        ga.classify_continuous_raster(rad_raster, gru_raster, rad_classif_trigger, rad_bins, 
                                      rad_class_raster, rad_value_raster, nodatavalue)        
        # (3) define hru discretization files
        raster_list = [gru_raster, dem_class_raster, rad_class_raster, lc_class_raster]
        raster_fieldname_list = [gruNo_field, 'elevClass', 'radClass', 'lcClass']

     # level 4: radiation bands are generated based on level2 HRU, add radiation bands to level2 HRU
    elif level == 4:
        # (1) classify elevation raster per gru
        ga.classify_continuous_raster(dem_crop, gru_raster, dem_classif_trigger, dem_bins, 
                                      dem_class_raster, dem_value_raster, nodatavalue)        
        # (2) classify radiation raster per level2 hru
        hru_lev2_raster = os.path.join(case_dir, 'hru'+'_lev' + str(2)+'_elmn.tif')
        ga.classify_continuous_raster(rad_raster, hru_lev2_raster, rad_classif_trigger, rad_bins, 
                                      rad_class_raster, rad_value_raster, nodatavalue)        
        # (3) define hru discretization files
        raster_list = [gru_raster, dem_class_raster, rad_class_raster, lc_class_raster]
        raster_fieldname_list = [gruNo_field, 'elevClass', 'radClass', 'lcClass']

#         # (2) define hru discretization files
#         raster_list = [gru_raster, hru_lev2_raster, rad_class_raster]
#         raster_fieldname_list = [gruNo_field, 'hruLev2', 'radClass']

    # --- PART 2. genearte HRU based on gru and elevation class ---
    print('PART 2. genearte HRU')
    ga.define_hru(raster_list, raster_fieldname_list, gru_raster, gru_corr_txt, gruNo_field, gruName_field,
                  nodatavalue, hru_raster, hru_vector, hruNo_field, hruNo_field_dtype, hruName_field)
#     ga.plot_vector(hru_vector, hruName_field) # quick plot for check

    # --- PART 3. calculate zonal area ---
    print('PART 3. calculate zonal area')
    in_gpd = gpd.read_file(hru_vector)
    in_gpd['areaSqm'] = in_gpd.area
    in_gpd.to_file(hru_vector)

    # --- PART 4. eliminate small HRUs ---
    print('PART 4. eliminate small HRUs')
    # method 2: change HRU attribute to its' largest neighbor's HRU
    ga.eliminate_small_hrus_neighbor(hru_vector, hru_thld_type, hru_thld, gruNo_field, gruName_field, hruNo_field, hruNo_field_dtype, 
                                     hruName_field, hruArea_field, raster_fieldname_list, refraster, hru_vector_elmn, hru_raster_elmn,
                                     nodatavalue)

    # --- PART 5. calculate zonal statistics ---
    print('PART 5. calculate zonal statistics ')
    for invector in [hru_vector,hru_vector_elmn]:
        
        # (1) define invector dependent files 
        invector_field = hruNo_field
        invector_field_dtype = hruNo_field_dtype
        attrb_elev = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_elevation.tif')        
        attrb_slp = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_slope.tif')        
        attrb_asp = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_aspect.tif')        
        attrb_lc = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_landcover.tif')        
        attrb_soil = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_soil.tif')        
        
        # (2) elevation zonal statistics 
        ga.zonal_statistic(dem_crop, invector, invector_field, invector_field_dtype, refraster, 'mean', attrb_elev,
                           nodatavalue, output_column_prefix='elev')

        # (3) slope zonal statistics 
        ga.zonal_statistic(slope_raster, invector, invector_field, invector_field_dtype, refraster, 'mean', attrb_slp,
                           nodatavalue, output_column_prefix='slope')

        # (4) aspect zonal statistics 
        ga.zonal_statistic(aspect_raster, invector, invector_field, invector_field_dtype, refraster, 'mean_aspect', attrb_asp,
                           nodatavalue, output_column_prefix='aspect')

        # (5) landcover zonal statistics 
        ga.zonal_statistic(lc_crop_resample, invector, invector_field, invector_field_dtype, refraster, 'mode', attrb_lc,
                           nodatavalue, output_column_prefix='vegType')        
        
        # (6) soil zonal statistics 
        ga.zonal_statistic(soil_crop, invector, invector_field, invector_field_dtype, refraster, 'mode', attrb_soil,
                           nodatavalue, output_column_prefix='soilType')        
        
        # -------- post-process attributes for SUMMA ---------
        # (7) landcover and soil types 
        # convert landcover int to [1,17] range 
        # change soilType from float to int (because source soilType is float)
        in_gpd = gpd.read_file(invector)
        in_gpd['vegType'] = in_gpd['vegType']+1
        in_gpd['soilType'] = in_gpd['soilType'].astype('int')
        in_gpd.to_file(invector)
        
        # (8) convert landcover int to string for easy understanding
        lcClass_list = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 255]
        lcValue_list = ['Evergreen needleleaf forests', 'Evergreen broadleaf forests', 'Deciduous needleleaf forests',
                        'Deciduous broadleaf forests', 'Mixed forests', 'Closed shrublands', 'Open shrublands', 
                        'Woody savannas', 'Savannas', 'Grasslands', 'Permanent wetlands', 'Croplands', 
                        'Urban and built-up lands', 'Cropland/natural vegetation mosaics', 'Snow and ice', 
                        'Barren', 'Water bodies', 'None']
        in_gpd = gpd.read_file(invector)
        in_gpd['landcover'] = ""
        for irow, row in in_gpd.iterrows():
            lcClass = in_gpd.loc[irow,'vegType'] 
            lcValue=lcValue_list[lcClass_list.index(lcClass)]
            in_gpd.at[irow,'landcover'] = lcValue
        in_gpd['landcover'] = in_gpd['landcover'].astype('str')
        in_gpd.to_file(invector)

        # (9) convert ROSETTA soil to STAS and add string for easy understanding
        soilClass_list = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        soilValue_list = ['OTHER(land-ice)', 'CLAY', 'CLAY LOAM', 'LOAM', 'LOAMY SAND', 'SAND', 'SANDY CLAY', 
                          'SANDY CLAY LOAM', 'SANDY LOAM', 'SILT','SILTY CLAY', 'SILTY CLAY LOAM', 'SILT LOAM']
        soilClass_list_STAS = [16, 12, 9, 6, 2, 1, 10, 7, 3, 5, 11, 8, 4]
        
        in_gpd = gpd.read_file(invector)
        in_gpd['soilROSETTA'] = in_gpd['soilType']
        in_gpd['soilSTAS'] = ""
        in_gpd['soil'] = ""
        for irow, row in in_gpd.iterrows():
            
            soilClass = in_gpd.loc[irow,'soilType'] 
            if soilClass==0:
                lcClass = in_gpd.loc[irow,'vegType'] 
                print('hruNo = %d, soilType_ROSETTA = 0, and vegType = %s.'%(in_gpd.loc[irow,'hruNo'],lcClass))
            
            soilValue=soilValue_list[soilClass_list.index(soilClass)]
            soilClass_STAS=soilClass_list_STAS[soilClass_list.index(soilClass)]
            
            in_gpd.at[irow,'soil'] = soilValue
            in_gpd.at[irow,'soilSTAS'] = soilClass_STAS
            
        in_gpd['soil'] = in_gpd['soil'].astype('str')
        in_gpd['soilSTAS'] = in_gpd['soilSTAS'].astype('int')
        in_gpd['soilType'] = in_gpd['soilSTAS']
        in_gpd = in_gpd.drop(columns=['soilSTAS'])
        in_gpd.to_file(invector)

        # (10) convert slope to tan_slope 
        in_gpd = gpd.read_file(invector)
        in_gpd['tan_slope'] = np.tan(np.radians(in_gpd['slope']))
        in_gpd.to_file(invector)

        # (11) calculate contourLength (meter)
        # assuming the hru area is a circle and taking the radius as contourLength.
        in_gpd = gpd.read_file(invector)
        in_gpd['contourLength'] = np.power(in_gpd['areaSqm']/np.pi,0.5)
        in_gpd.to_file(invector)

        # (12) calculate centroid lat/lon (degree)
        def getXY(pt):
            return (pt.x, pt.y)
        in_gpd = gpd.read_file(invector)
        in_gpd_wgs84 = in_gpd.copy()
        in_gpd_wgs84 = in_gpd_wgs84.to_crs(epsg=4326) #"EPSG:4326"
        centroidseries = in_gpd_wgs84['geometry'].centroid
        in_gpd['longitude'],in_gpd['latitude']=[list(t) for t in zip(*map(getXY, centroidseries))]
        in_gpd.to_file(invector)

    # --- PART 6. save as gpkg ---
    invector_gpkg = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'.gpkg')
    in_gpd = gpd.read_file(invector)
    in_gpd.to_file(invector_gpkg, driver="GPKG")
    
print('Done')

--- Complexity level 0 ---
PART 1. define files
PART 2. genearte HRU


100%|██████████| 18/18 [00:00<00:00, 546.79it/s]

PART 3. calculate zonal area
PART 4. eliminate small HRUs


PART 5. calculate zonal statistics 


<ipython-input-13-5521acb113e1>:202: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidseries = in_gpd_wgs84['geometry'].centroid


...

PART 4. eliminate small HRUs


100%|██████████| 18/18 [00:08<00:00,  2.08it/s]


PART 5. calculate zonal statistics 


<ipython-input-13-5521acb113e1>:202: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidseries = in_gpd_wgs84['geometry'].centroid


Done


### Section 4. Calculate GRU attributes (if needed) ###

In [14]:
invector = gru_shp_prj
invector_field = gruNo_field
invector_field_dtype = gruNo_field_dtype

attrb_elev = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_elevation.tif')        
attrb_slp = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_slope.tif')        
attrb_asp = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_aspect.tif')        
attrb_lc = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_landcover.tif')        
attrb_soil = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb_soil.tif')        

outvector = os.path.join(case_dir, os.path.basename(invector).split('.shp')[0]+'_attrb.gpkg')     
in_gpd = gpd.read_file(invector)
in_gpd.to_file(outvector, driver="GPKG")
invector = outvector # avoid process gru_shp_prj. Work on gpkg.

# (1) calculate zonal area ---
in_gpd = gpd.read_file(invector)
in_gpd['areaSqm'] = in_gpd.area
in_gpd.to_file(invector)

# (2) elevation zonal statistics 
ga.zonal_statistic(dem_crop, invector, invector_field, invector_field_dtype, refraster, 'mean', attrb_elev,
                   nodatavalue, output_column_prefix='elev')

# (3) slope zonal statistics 
ga.zonal_statistic(slope_raster, invector, invector_field, invector_field_dtype, refraster, 'mean', attrb_slp,
                   nodatavalue, output_column_prefix='slope')

# (4) aspect zonal statistics 
ga.zonal_statistic(aspect_raster, invector, invector_field, invector_field_dtype, refraster, 'mean_aspect', attrb_asp,
                   nodatavalue, output_column_prefix='aspect')

# (5) landcover zonal statistics 
ga.zonal_statistic(lc_crop_resample, invector, invector_field, invector_field_dtype, refraster, 'mode', attrb_lc,
                   nodatavalue, output_column_prefix='vegType')        

# (6) soil zonal statistics 
ga.zonal_statistic(soil_crop, invector, invector_field, invector_field_dtype, refraster, 'mode', attrb_soil,
                   nodatavalue, output_column_prefix='soilType')        

# -------- post-process attributes for SUMMA ---------
# (7) landcover and soil types 
# convert landcover int to [1,17] range 
# change soilType from float to int (because source soilType is float)
in_gpd = gpd.read_file(invector)
in_gpd['vegType'] = in_gpd['vegType']+1
in_gpd['soilType'] = in_gpd['soilType'].astype('int')
in_gpd.to_file(invector)

# (8) convert landcover int to string for easy understanding
lcClass_list = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 255]
lcValue_list = ['Evergreen needleleaf forests', 'Evergreen broadleaf forests', 'Deciduous needleleaf forests',
                'Deciduous broadleaf forests', 'Mixed forests', 'Closed shrublands', 'Open shrublands', 
                'Woody savannas', 'Savannas', 'Grasslands', 'Permanent wetlands', 'Croplands', 
                'Urban and built-up lands', 'Cropland/natural vegetation mosaics', 'Snow and ice', 
                'Barren', 'Water bodies', 'None']
in_gpd = gpd.read_file(invector)
in_gpd['landcover'] = ""
for irow, row in in_gpd.iterrows():
    lcClass = in_gpd.loc[irow,'vegType'] 
    lcValue=lcValue_list[lcClass_list.index(lcClass)]
    in_gpd.at[irow,'landcover'] = lcValue
in_gpd['landcover'] = in_gpd['landcover'].astype('str')
in_gpd.to_file(invector)

# (9) convert ROSETTA soil to STAS and add string for easy understanding
soilClass_list = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
soilValue_list = ['OTHER(land-ice)', 'CLAY', 'CLAY LOAM', 'LOAM', 'LOAMY SAND', 'SAND', 'SANDY CLAY', 
                  'SANDY CLAY LOAM', 'SANDY LOAM', 'SILT','SILTY CLAY', 'SILTY CLAY LOAM', 'SILT LOAM']
soilClass_list_STAS = [16, 12, 9, 6, 2, 1, 10, 7, 3, 5, 11, 8, 4]

in_gpd = gpd.read_file(invector)
in_gpd['soilROSETTA'] = in_gpd['soilType']
in_gpd['soilSTAS'] = ""
in_gpd['soil'] = ""
for irow, row in in_gpd.iterrows():

    soilClass = in_gpd.loc[irow,'soilType'] 
    if soilClass==0:
        lcClass = in_gpd.loc[irow,'vegType'] 
        print('hruNo = %d, soilType_ROSETTA = 0, and vegType = %s.'%(in_gpd.loc[irow,'hruNo'],lcClass))

    soilValue=soilValue_list[soilClass_list.index(soilClass)]
    soilClass_STAS=soilClass_list_STAS[soilClass_list.index(soilClass)]

    in_gpd.at[irow,'soil'] = soilValue
    in_gpd.at[irow,'soilSTAS'] = soilClass_STAS

in_gpd['soil'] = in_gpd['soil'].astype('str')
in_gpd['soilSTAS'] = in_gpd['soilSTAS'].astype('int')
in_gpd['soilType'] = in_gpd['soilSTAS']
in_gpd = in_gpd.drop(columns=['soilSTAS'])
in_gpd.to_file(invector)

# (10) convert slope to tan_slope 
in_gpd = gpd.read_file(invector)
in_gpd['tan_slope'] = np.tan(np.radians(in_gpd['slope']))
in_gpd.to_file(invector)

# (11) calculate contourLength (meter)
# assuming the hru area is a circle and taking the radius as contourLength.
in_gpd = gpd.read_file(invector)
in_gpd['contourLength'] = np.power(in_gpd['areaSqm']/np.pi,0.5)
in_gpd.to_file(invector)

# (12) calculate centroid lat/lon (degree)
def getXY(pt):
    return (pt.x, pt.y)
in_gpd = gpd.read_file(invector)
in_gpd_wgs84 = in_gpd.copy()
in_gpd_wgs84 = in_gpd_wgs84.to_crs(epsg=4326) #"EPSG:4326"
centroidseries = in_gpd_wgs84['geometry'].centroid
in_gpd['longitude'],in_gpd['latitude']=[list(t) for t in zip(*map(getXY, centroidseries))]
in_gpd.to_file(invector)

print('Done')

Done


<ipython-input-14-6a40edc99525>:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidseries = in_gpd_wgs84['geometry'].centroid
